In [2]:
import xarray as xr
import geocat.comp as geocomp
import copy
import pathlib as pl
import time
from workflow.scripts.utils import calc_relative_change, calc_abs_change


In [22]:
path_ctrl = snakemake.input.path_ctrl
path_exp = snakemake.input.path_exp
vname = snakemake.wildcards.variable
params = snakemake.params
time_slice = params.get('time_slice', slice(5,None))
cmap = params.get('cmap', None)
nlevels = params.get('nlevels', 21)

In [23]:
ds_ctrl = xr.open_dataset(path_ctrl[0])
ds_exp = xr.open_dataset(path_exp[0])
delta = calc_abs_change(ds_ctrl,ds_exp,time_slice=time_slice,vertical_sum=False)
#delta = ds_exp[vname]-ds_ctrl[vname]
if 'ap' in ds_ctrl.data_vars:
    ds_ctrl = ds_ctrl.rename_vars({'ap': 'a'})
    ds_exp = ds_exp.rename_vars({'ap': 'a'})

if 'lev_bounds' in ds_ctrl.data_vars:
    ds_ctrl = ds_ctrl.rename_vars({'lev_bounds': 'lev_bnds'})
    ds_exp = ds_exp.rename_vars({'lev_bounds': 'lev_bnds'})


In [24]:
def get_surface_pressure_field(ds_ctrl, ds_exp):
    path0_ctrl = pl.Path('/'.join(ds_ctrl.source.split('/')[:-4])).joinpath('ps')
    path0_exp = pl.Path('/'.join(ds_exp.source.split('/')[:-4])).joinpath('ps')
    return path0_ctrl, path0_exp
    

path0_, path_exp = get_surface_pressure_field(ds_ctrl, ds_exp)

In [26]:
ds_exp

In [21]:
if 'plev' not in ds_exp.coords:
#     if ds_ctrl.source_id in ['IPSL-CM6A-LR-INCA','GISS-E2-1-G', '']:
#         delta = delta.rename_dims({'lev':'plev'})
#         delta = delta.rename_vars({'lev':'plev'})
    
    if ds_ctrl.cf['Z'].formula == 'p = a*p0 + b*ps':
        delta = geocomp.interpolation.interp_hybrid_to_pressure(data  =delta[vname], 
                                                        ps        =ds_ctrl['ps'].sel(time=delta.time), 
                                                        hyam      =ds_ctrl['a'], 
                                                        hybm      =ds_ctrl['b'], 
                                                        p0        =ds_ctrl.get('p0', 100000.0))
    elif ds_ctrl.cf['Z'].formula == 'p = ap + b*ps':
        delta = geocomp.interpolation.interp_hybrid_to_pressure(data  =delta[vname], 
                                                        ps        =ds_ctrl['ps'].sel(time=delta.time), 
                                                        hyam      =ds_ctrl['a'], 
                                                        hybm      =ds_ctrl['b'], 
                                                        p0        =ds_ctrl.get('p0', 1))

        
    else:
        print('formula not reconized')

In [12]:
delta = delta.compute()

In [13]:
if isinstance(delta,xr.DataArray):
    delta.attrs['long_name'] =delta.attrs['long_name']
    delta = delta.to_dataset(name=vname)

In [14]:
delta.attrs = copy.deepcopy(ds_ctrl.attrs)

In [15]:
delta.attrs['history'] = delta.attrs['history'] + f'@{time.ctime()} converted to standard pressure levels {pl.Path.cwd().parts[-1]}'
delta.to_netcdf(snakemake.output.outpath)